In [17]:
!bq --location=US mk --dataset covid19_data_analysis

BigQuery error in mk operation: Dataset 'sunny-
advantage-266802:covid19_data_analysis' already exists.


In [18]:
!bq \
    --location=US \
    load \
    --autodetect \
    --skip_leading_rows=1 \
    --source_format=CSV \
    {dataset_id}.GDP \
    'gs://the-data-miners-cdc-bucket/the-data-miners-cdc-folder/GDP_2018.csv'

Waiting on bqjob_r41aa282b1f52ea0d_00000171bd63cb87_1 ... (1s) Current status: DONE   


In [26]:
%%bigquery
select distinct country_territory_code, confirmed_cases, deaths from covid19_data_analysis.covid_19_geographic_distribution_worldwide
where date = '2020-04-26' and country_territory_code is not Null

,country_territory_code,confirmed_cases,deaths
0,AND,733,40
1,ARE,9813,64
2,AFG,1463,47
3,ATG,24,3
4,ALB,712,27
...,...,...,...
196,XKX,731,20
197,YEM,1,0
198,ZAF,4361,86
199,ZMB,84,3


In [29]:
%%bigquery
create table covid19_data_analysis.Country_GDP as
select distinct string_field_0 as Country, string_field_4 as GDP from covid19_data_analysis.GDP
where string_field_0 is not null

""


In [45]:
%%bigquery
select * from covid19_data_analysis.Country_GDP

,Country,GDP
0,TUV,43
1,NRU,126
2,KIR,188
3,MHL,221
4,PLW,284
...,...,...
224,ECS,"23,068,403"
225,UMC,"24,446,133"
226,EAS,"25,942,413"
227,HIC,"54,205,741"


In [30]:
%%bigquery
create table covid19_data_analysis.Country_cases_deaths as
select distinct country_territory_code, confirmed_cases, deaths from covid19_data_analysis.covid_19_geographic_distribution_worldwide
where date = '2020-04-26' and country_territory_code is not Null

""


In [46]:
%%bigquery
select * from covid19_data_analysis.Country_cases_deaths

,country_territory_code,confirmed_cases,deaths
0,DMA,16,0
1,UGA,75,0
2,TLS,24,0
3,FRO,187,0
4,VCT,14,0
...,...,...,...
196,UKR,8617,209
197,CZE,7352,218
198,COL,5142,233
199,PHL,7294,494


In [61]:
%%bigquery
create table covid19_data_analysis.Combined_GDP_cases_death_table as
select Country_GDP.Country, Country_GDP.GDP, Country_cases_deaths.confirmed_cases, Country_cases_deaths.deaths from covid19_data_analysis.Country_GDP join covid19_data_analysis.Country_cases_deaths
on Country_cases_deaths.country_territory_code = Country_GDP.Country
where Country_cases_deaths.country_territory_code is not null and Country_GDP.GDP is not null
order by Country_cases_deaths.confirmed_cases DESC

""


In [77]:
%%bigquery
select * from covid19_data_analysis.Combined_GDP_cases_death_table
limit 10

,Country,GDP,confirmed_cases,deaths
0,USA,"20,544,343",939053,53189
1,ITA,"2,083,864",195351,26384
2,DEU,"3,947,620",154175,5640
3,GBR,"2,855,297",148377,20319
4,FRA,"2,777,535",124114,22614
5,TUR,"771,350",107773,2706
6,IRN,"454,013",89328,5650
7,CHN,"13,608,152",83909,4636
8,RUS,"1,657,555",74588,681
9,BRA,"1,868,626",58509,4016


In [80]:
%%bigquery
create table covid19_data_analysis.Combined_GDP_cases_death_table_final as
select Country_GDP.Country, replace(Country_GDP.GDP,',','') GDP, Country_cases_deaths.confirmed_cases, Country_cases_deaths.deaths from covid19_data_analysis.Country_GDP join covid19_data_analysis.Country_cases_deaths
on Country_cases_deaths.country_territory_code = Country_GDP.Country
where Country_cases_deaths.country_territory_code is not null and Country_GDP.GDP is not null
order by Country_cases_deaths.confirmed_cases DESC

""


In [85]:
%%bigquery
select * from covid19_data_analysis.Combined_GDP_cases_death_table_final
limit 10

,Country,GDP,confirmed_cases,deaths
0,USA,20544343,939053,53189
1,ITA,2083864,195351,26384
2,DEU,3947620,154175,5640
3,GBR,2855297,148377,20319
4,FRA,2777535,124114,22614
5,TUR,771350,107773,2706
6,IRN,454013,89328,5650
7,CHN,13608152,83909,4636
8,RUS,1657555,74588,681
9,BRA,1868626,58509,4016


In [90]:
%%bigquery
select Country, CAST(GDP AS INT64)GDP, confirmed_cases, deaths  from covid19_data_analysis.Combined_GDP_cases_death_table_final
limit 10

,Country,GDP,confirmed_cases,deaths
0,USA,20544343,939053,53189
1,ITA,2083864,195351,26384
2,DEU,3947620,154175,5640
3,GBR,2855297,148377,20319
4,FRA,2777535,124114,22614
5,TUR,771350,107773,2706
6,IRN,454013,89328,5650
7,CHN,13608152,83909,4636
8,RUS,1657555,74588,681
9,BRA,1868626,58509,4016


In [93]:
%%bigquery
create or replace view covid19_data_analysis.Relationship_between_GDP_and_COVID19 as
select Country, CAST(GDP AS INT64)GDP, confirmed_cases, deaths 
from `sunny-advantage-266802.covid19_data_analysis.Combined_GDP_cases_death_table_final`

""
